In [16]:
from pyproj import Proj, Transformer
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# 데이터 불러오기
df = pd.read_csv("서울시 휴게음식점 인허가 정보.csv", encoding="EUC-KR", encoding_errors="replace", low_memory=False)

# 중부원점TM (EPSG:2097) → WGS84 (EPSG:4326) 변환기 (방식 변경)
tm_proj = Proj("epsg:2097")
wgs84_proj = Proj("epsg:4326")
transformer = Transformer.from_proj(tm_proj, wgs84_proj, always_xy=True)

# 중부원점TM → WGS84 위도/경도 변환 (NaN 값 처리 + 좌표 변환 방식 개선)
df["경도"], df["위도"] = zip(*df.apply(
    lambda x: transformer.transform(x["좌표정보(X)"], x["좌표정보(Y)"])
    if pd.notna(x["좌표정보(X)"]) and pd.notna(x["좌표정보(Y)"]) else (None, None), axis=1
))

# 영업 중인 지점만 추출
df = df[df["상세영업상태명"] == "영업"]
new_df = df[["경도", "위도", "업태구분명"]].dropna().reset_index(drop=True)

# GeoJSON 파일 읽기 (성능 최적화: 한 번만 로드)
geojson_file = "hangjeongdong_서울특별시.geojson"
gdf = gpd.read_file(geojson_file)

# 행정동 정보 추출 함수 (속도 개선)
def find_administrative_dong(latitude, longitude, gdf):
    point = Point(longitude, latitude)
    result = gdf[gdf.contains(point)]
    return result.iloc[0]["adm_nm"] if not result.empty else "해당 좌표는 어떤 행정동에도 속하지 않습니다."

# 행정동을 찾는 함수를 적용
new_df["행정동"] = new_df.apply(lambda x: find_administrative_dong(x["위도"], x["경도"], gdf), axis=1)

# 결과 출력
full_data = new_df[new_df["행정동"] != "해당 좌표는 어떤 행정동에도 속하지 않습니다."]
print(full_data)

               경도         위도     업태구분명                    행정동
0      126.889031  37.496476    일반조리판매         서울특별시 구로구 구로5동
1      127.037165  37.528562       백화점         서울특별시 강남구 압구정동
2      127.017925  37.583142       커피숍          서울특별시 성북구 보문동
3      126.942167  37.479251       커피숍          서울특별시 관악구 청룡동
4      126.984166  37.569075  기타 휴게음식점  서울특별시 종로구 종로1·2·3·4가동
...           ...        ...       ...                    ...
36745  126.858366  37.554653       커피숍         서울특별시 강서구 등촌1동
36746  127.042357  37.503812       커피숍         서울특별시 강남구 역삼1동
36747  127.007739  37.554250  기타 휴게음식점           서울특별시 중구 다산동
36748  127.086858  37.505313  기타 휴게음식점         서울특별시 송파구 잠실본동
36749  126.844148  37.530731       커피숍         서울특별시 강서구 화곡1동

[36573 rows x 4 columns]


In [22]:
final_data = pd.pivot_table(full_data, index="업태구분명", columns="행정동", aggfunc="size", fill_value=0)

In [26]:
final_data.to_excel("cafe_data.xlsx")